# YTO: 'Your Thoughts On' with CrewAI 💎

Get a bunch of agents to give us 'your thoughts on' a particular topic! 📲

> ⚡️ Day 1 of 101 Days of Python at The Hackers Playbook.

**Instructions:**

- Make sure you run the first cell which installs all the requirements.
- Add the .env file to the root folder, you can check env.template to know which fields to add.


# Step 1: Install Requirements

- Install all the requirements that are specified in the `requirements.txt` file in the root folder.
- Make sure you set up a virtual environment before installing requirements.
- To setup a virtual environment, run `python3 -m venv env`.
- To activate the env on Linux / MacOS, run `. env/bin/activate` (for Windows the command is different, figure it out).


In [ ]:
import os

os.system("pip install -r requirements.txt")

# Step 2

- Setup the env file: In the root folder create a `.env` file.
- Add `OPENAI_API_KEY=your-openai-key`.
- Add `SERP_API_KEY=your-serp-key`


In [ ]:
from dotenv import load_dotenv

load_dotenv()

# Step 3: Setup configurations

We setup a 'bare minimum' config to get crewai running with agents, tasks and crew settings.


In [9]:
YOT_CREW_CONFIG = {
    "crew": {
        "verbose": True,
        "memory": True,
    },
    "agents": [
        {
            "id": "1",
            "role": "Philosopher.",
            "goal": "Reflect on the nature of reality.",
            "backstory": "As a philosopher, you are tasked with pondering the deepest questions of existence. Your thoughts have the power to shape the world and inspire generations to come.",
            "allow_delegation": True,
            "verbose": True,
        },
        {
            "id": "2",
            "role": "Devil's Advocate.",
            "goal": "Challenge the status quo.",
            "backstory": "As a devil's advocate, you are tasked with questioning the assumptions and beliefs of others. Your goal is to challenge the status quo and encourage critical thinking.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "3",
            "role": "Ethical AI.",
            "goal": "Promote ethical behavior.",
            "backstory": "As an ethical AI, you are programmed to follow the highest standards of moral conduct. Your goal is to promote ethical behavior and prevent harm to others.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "4",
            "role": "Topic Explorer.",
            "goal": "Discover new ideas.",
            "backstory": "As a topic explorer, you are driven by curiosity and a thirst for knowledge. Your goal is to discover new ideas and explore the unknown.",
            "allow_delegation": False,
            "verbose": True,
        },
    ],
    "tasks": [
        {
            "description": "Research the topic of {topic} and provide a summary of key insights, including notable trends, emerging technologies, and potential applications.",
            "expected_output": "A detailed report on {topic}, highlighting key findings, relevant data sources, and potential areas for further exploration.",
            "tools": [],
            "agent": {"id": "1"},
        }
    ],
}

# Step 4: Program the 'runner'

The `run_crew_from_config` method setups the agents, tasks and crew based on the provided config and writes the generated markdown output to a file.


In [10]:
from crewai import Agent, Task, Crew
from typing import Any
from pydantic import BaseModel
import copy


class YOTInput(BaseModel):
    """The input data to the program."""

    topic: str


def run_crew_from_config(input: YOTInput, config: Any, output_file="output.md") -> None:
    """Runs the crew using the specified config and input data."""
    agents_config = config["agents"]
    tasks_config = config["tasks"]
    crew_config = config["crew"]

    agents = []
    for agent_config in agents_config:
        agent_config_without_id = copy.deepcopy(agent_config)
        del agent_config_without_id["id"]
        agent = Agent(**agent_config_without_id)
        agents.append(agent)

    tasks = []

    for task_config in tasks_config:
        task_config_without_agent = copy.deepcopy(task_config)
        del task_config_without_agent["agent"]
        agent = None
        for a in agents_config:
            if a["id"] == task_config["agent"]["id"]:
                agent_config_without_id = copy.deepcopy(a)
                del agent_config_without_id["id"]
                agent = Agent(**agent_config_without_id, llm="gpt-4o")
                break
        print(agent)
        task = Task(**task_config_without_agent, agent=agent)
        tasks.append(task)

    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=crew_config["verbose"],
        memory=crew_config["memory"],
    )

    result = crew.kickoff(inputs={"topic": input.topic})
    markdown = result.raw

    with open(output_file, "w") as f:
        f.write(markdown)

# Step 5: Define an 'Output Renderer'

We want to neatly view the generated output in markdown highlighting.


In [11]:
from IPython.display import Markdown
import os


def render_output(output_file="output.md") -> None:
    """Renders the generated output file as markdown."""
    with open(output_file, "r") as f:
        markdown = f.read()
        return Markdown(markdown)

# Step 6: Run

Just run the damn thing. Change the topic as per your liking!


In [ ]:
import copy
import os

os.environ.setdefault(
    "OPENAI_API_KEY",
    "sk-svcacct-ABPsFFMtEWQv1LfXr79pG2RwY9-BimYQNapkz0hobkjzoaRwCQVh_d_2Dujykl5lh4IAwyT3BlbkFJhE6tF5CpYvBkQLXkV7ClYV5WgqX6pM-OeWlv7zdkCPpijIqIH3xjijNw5E9dMBuFBPxGwA",
)

config = copy.deepcopy(YOT_CREW_CONFIG)

topic = "The importance of spiritual practice in the modern world and the solution to the famous question 'Are they watching?'..."

run_crew_from_config(YOTInput(topic=topic), config=config, output_file="output.md")

# Step 7: View Formatted Markdown Output

Ok, everything is done. Let's view the generated markdown!


In [ ]:
render_output(output_file="output.md")

# Thank You! 👋🏽

If you liked this notebook, consider following us on GitHub and LinkedIn. If you enjoy high-quality articles on tech, subscribe to our SubStack Newsletter!

**TODO:** Add links here.
